In [501]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

In [502]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")
history = df_all

In [503]:
order18 = pd.read_csv('data/running_order_j2018_ju.tsv', delimiter="\t")

In [504]:
#paces = df.as_matrix(["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
paces = history[["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"]]
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
history = history.assign(log_means=pd.Series(means).values)
history = history.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [505]:
# Estimate values for all, but only use them if no history is available
with_history = history[np.isfinite(history.pace_2)]
x = with_history.mean_team_id.values
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)

estimated_log_means = log_means_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)

estimated_log_stdevs = log_stdevs_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_stdevs=estimated_log_stdevs)
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs
0,5,Halden SK 1,Halden SK,1,11.0,Emil Wingstedt,1.944872,0.085008
1,5,Halden SK 1,Halden SK,2,11.9,Fredrik Eliasson,1.944872,0.085008
2,5,Halden SK 1,Halden SK,3,12.8,Thomas Natvig Årstad,1.944872,0.085008
3,5,Halden SK 1,Halden SK,4,8.7,Alexander Chepelin,1.944872,0.085008
4,5,Halden SK 1,Halden SK,5,8.7,Riccardo Scalet,1.944872,0.085008


In [506]:
# Combine history with 2018 runners 
no_history_row = pd.DataFrame([[0,0]], columns=["log_means", "log_stdevs"])
def get_history_row(running_order_row):
    name = running_order_row["name"].lower()
    
    by_name = history[history['name'] == name]
    by_name_and_colon = history[history['name'].str.contains(name + ":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = running_order_row["team_base_name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    if(len(runners) == 0):
        return no_history_row
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(running_order_row):
    history_row = get_history_row(running_order_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    log_means = history_row.log_means.values[0]
    log_stdevs = history_row.log_stdevs.values[0]
    return pd.Series({"history_log_means": log_means, "history_log_stdevs": log_stdevs})

#order18 = order18[order18['team'].str.contains("Reak") | order18['team'].str.contains("Puskasil") | order18['team'].str.contains("Rastihaukat")]
estimate_params = order18.apply(lambda row: get_estimate_params(row), axis=1)
order18 = order18.assign(history_log_means = estimate_params.history_log_means)
order18 = order18.assign(history_log_stdevs = estimate_params.history_log_stdevs)

In [507]:
order18['log_means'] = np.where(np.isfinite(order18["history_log_means"]) & order18["history_log_means"] > 0, order18["history_log_means"], order18["estimated_log_means"])
order18['log_stdevs'] = np.where(np.isfinite(order18["history_log_stdevs"]) & order18["history_log_stdevs"] > 0, order18["history_log_stdevs"], order18["estimated_log_stdevs"])
order18.head()


,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs
0,5,Halden SK 1,Halden SK,1,11.0,Emil Wingstedt,1.944872,0.085008,1.763072,0.025436,1.763072,0.025436
1,5,Halden SK 1,Halden SK,2,11.9,Fredrik Eliasson,1.944872,0.085008,1.842846,0.093588,1.842846,0.093588
2,5,Halden SK 1,Halden SK,3,12.8,Thomas Natvig Årstad,1.944872,0.085008,1.797910,0.103600,1.797910,0.103600
3,5,Halden SK 1,Halden SK,4,8.7,Alexander Chepelin,1.944872,0.085008,1.929038,0.069687,1.929038,0.069687
4,5,Halden SK 1,Halden SK,5,8.7,Riccardo Scalet,1.944872,0.085008,1.958229,0.119586,1.958229,0.119586


In [508]:
# Calculate personal estimates
# s = sigma and scale = exp(mu).

log_means = np.exp(order18['log_means']) 
log_stdevs = order18['log_stdevs']

intervals95 = lognorm.interval(0.95, s = log_stdevs, scale = log_means)
means = lognorm.mean(s = log_stdevs, scale = log_means)
medians = lognorm.median(s = log_stdevs, scale = log_means)

In [509]:
order18 = order18.assign(ind_95_start = pd.Series(intervals95[0] * order18.leg_dist).values)
order18 = order18.assign(ind_95_end = pd.Series(intervals95[1] * order18.leg_dist).values)
order18 = order18.assign(ind_mean = pd.Series(means * order18.leg_dist).values)
order18 = order18.assign(ind_median = pd.Series(medians * order18.leg_dist).values)

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs,ind_95_start,ind_95_end,ind_mean,ind_median
1923,563,Kalevan Rasti 6,Kalevan Rasti,1,11.0,Jarkko Tiihonen,2.186753,0.094369,2.422980,0.023359,2.422980,0.023359,118.521281,129.886128,124.107503,124.073649
1924,563,Kalevan Rasti 6,Kalevan Rasti,2,11.9,Pasi Hurskainen,2.186753,0.094369,0.000000,0.000000,2.186753,0.094369,88.087522,127.517201,106.457288,105.984311
1925,563,Kalevan Rasti 6,Kalevan Rasti,3,12.8,Pieti Liukka,2.186753,0.094369,2.137731,0.025822,2.137731,0.025822,103.189349,114.181161,108.582404,108.546210
1926,563,Kalevan Rasti 6,Kalevan Rasti,4,8.7,Jari Nurmi,2.186753,0.094369,2.518346,0.236270,2.518346,0.236270,67.937544,171.528447,111.005601,107.950088
1927,563,Kalevan Rasti 6,Kalevan Rasti,5,8.7,Rauno Asikainen,2.186753,0.094369,2.181253,0.174103,2.181253,0.174103,54.780841,108.398306,78.236199,77.059395


In [511]:
by_teams = order18.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                              \
leg                               1                           2   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                                                 \
leg                               3                           4   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                                                 \
leg                               5                           6   
team_id                                                           
5                       Halden SK 1                 Halden SK 1   
11                       OK Linne 1                  OK Linne 1   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima 1   
17                     IL Tyrving 1                IL Tyrving 1   
18                    Nydalens SK 1               Nydalens SK 1   

                                               team_base_name  \
leg                               7                         1   
team_id                                                         
5                       Halden SK 1                 Halden SK   
11                       OK Linne 1                  OK Linne   
14       Ikaalisten Nouseva-Voima 1  Ikaalisten Nouseva-Voima   
17                     IL Tyrving 1                IL Tyrving   
18                    Nydalens SK 1               Nydalens SK   

                                                                ...      \
leg                             2                         3     ...       
team_id                                                         ...       
5                       Halden SK                 Halden SK     ...       
11                       OK Linne                  OK Linne     ...       
14       Ikaalisten Nouseva-Voima  Ikaalisten Nouseva-Voima     ...       
17                     IL Tyrving                IL Tyrving     ...       
18                    Nydalens SK               Nydalens SK     ...       

          ind_mean                        ind_median                        \
leg              5          6           7          1          2          3   
team_id                                                                      
5        62.097304  75.794950   85.633899  64.133501  75.142332  77.273814   
11       56.721972  67.114796   92.568771  68.047577  75.630890  74.017768   
14       62.576465  66.628415  102.489651  70.737369  77.901033  84.792953   
17       57.333536  67.761076   84.976476  65.716365  72.341834  72.698226   
18       50.992857  65.046605   94.260852  69.885691  77.440836  76.428838   

                                                      
leg              4          5          6           7  
team_id                                               
5        59.881118  61.654866  75.521582   85.578863  
11       51.993243  55.593390  66.933450   92.321787  
14       54.158064  62.080109  66.525176  101.902216  
17       53.732789  57.079184  67.619799   84.945242  
18       55.973405  50.808000  64.810800   93.949004  

[5 rows x 98 columns]

In [512]:
# remove teams missing some runners
print(len(by_teams))
by_teams = by_teams[np.isfinite(by_teams.log_means[1]) & np.isfinite(by_teams.log_means[2]) & np.isfinite(by_teams.log_means[3]) & np.isfinite(by_teams.log_means[4]) & np.isfinite(by_teams.log_means[5]) & np.isfinite(by_teams.log_means[6]) & np.isfinite(by_teams.log_means[7])] 
print(len(by_teams))

1067
1045


In [513]:
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        if np.isnan(row["log_means"][i]):
            print(row["log_means"])
            print(row["name"])
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["log_stdevs"][i], scale = np.exp(row["log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team_id,,,,,,,,,,,,,,,,,,,,,
5,61.096118,126.038682,197.366621,255.745094,313.437920,386.470075,471.382098,67.378716,155.070741,239.496804,...,339.197981,414.974607,500.655807,64.164100,139.697916,217.472672,277.457955,339.525671,415.331256,501.025541
11,57.389099,126.713725,197.100568,248.550617,296.903008,362.883451,452.473145,81.314257,164.722695,242.057552,...,326.833899,394.044446,486.859605,68.400843,144.450779,218.723673,270.778435,327.624540,394.729631,487.289179
14,57.887516,131.101302,208.257755,261.589843,320.474625,385.892455,483.318189,86.430837,169.448286,264.063289,...,350.980131,417.703602,519.994126,71.069918,149.218957,234.564565,288.858848,351.438494,418.083531,520.573151
17,56.746221,127.659993,197.584416,248.551761,303.210807,368.947408,453.540091,76.010077,149.967587,225.295154,...,322.159016,389.867285,474.914818,65.925396,138.294402,211.136990,265.070666,322.371245,390.105739,475.057810
18,58.384073,127.711665,201.166051,255.527536,304.713368,367.784904,459.001672,83.860284,170.934788,251.067247,...,331.818988,396.873288,491.019364,70.264723,148.163015,224.905284,281.099880,332.141001,397.171344,491.394808


In [514]:
len(relay_estimates)

1045

In [515]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()


,team_id,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_base_name_1,team_base_name_2,...,ind_mean_5,ind_mean_6,ind_mean_7,ind_median_1,ind_median_2,ind_median_3,ind_median_4,ind_median_5,ind_median_6,ind_median_7
0,5,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK 1,Halden SK,Halden SK,...,62.097304,75.794950,85.633899,64.133501,75.142332,77.273814,59.881118,61.654866,75.521582,85.578863
1,11,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne 1,OK Linne,OK Linne,...,56.721972,67.114796,92.568771,68.047577,75.630890,74.017768,51.993243,55.593390,66.933450,92.321787
2,14,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima 1,Ikaalisten Nouseva-Voima,Ikaalisten Nouseva-Voima,...,62.576465,66.628415,102.489651,70.737369,77.901033,84.792953,54.158064,62.080109,66.525176,101.902216
3,17,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving 1,IL Tyrving,IL Tyrving,...,57.333536,67.761076,84.976476,65.716365,72.341834,72.698226,53.732789,57.079184,67.619799,84.945242
4,18,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK 1,Nydalens SK,Nydalens SK,...,50.992857,65.046605,94.260852,69.885691,77.440836,76.428838,55.973405,50.808000,64.810800,93.949004
5,26,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI 1,NTNUI,NTNUI,...,61.617233,72.004349,101.795724,68.011089,76.987816,80.864879,66.259730,61.393158,71.633572,101.693527
6,28,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri 1,Angelniemen Ankkuri,Angelniemen Ankkuri,...,58.784104,67.740436,108.455636,85.074000,81.385672,79.706644,55.722948,58.474682,67.436225,108.060920
7,32,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion 1,OK Orion,OK Orion,...,58.861257,76.690217,97.081819,78.620226,74.145082,81.015163,58.979693,58.646700,76.410671,96.672657
8,35,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland 1,OLV Baselland,OLV Baselland,...,57.324702,61.586819,86.896183,62.876000,76.362300,83.038686,53.400600,57.115500,61.538667,86.667308
9,37,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering 1,Järla Orientering,Järla Orientering,...,58.862402,78.894655,95.667554,69.616274,69.794621,84.531260,57.710983,58.602492,78.501248,95.300935


In [516]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [517]:
# Convert minutes to date and times
start_timestamp = pd.Timestamp(year = 2018, month = 6, day = 16, hour = 23)

for leg in range(1,8):
    estimates[f"fint_median_{leg}"] = pd.to_datetime(estimates[f"fin_median_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_start95_{leg}"] = pd.to_datetime(estimates[f"fin_start95_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_end95_{leg}"] = pd.to_datetime(estimates[f"fin_end95_{leg}"] * 60, unit = "s", origin= start_timestamp)


In [528]:
# Sort teams by estimated total time 
estimates = estimates.sort_values("fin_median_7")

estimates.to_csv('data/team_estimates_ju2018.tsv', sep="\t")

In [534]:
estimates[["team_1", "fin_median_7"]].head(10)

,team_1,fin_median_7
team_id,,
17,IL Tyrving 1,474.914818
35,OLV Baselland 1,481.893218
11,OK Linne 1,486.859605
18,Nydalens SK 1,491.019364
5,Halden SK 1,500.655807
37,Järla Orientering 1,515.559848
14,Ikaalisten Nouseva-Voima 1,519.994126
150,IL Tyrving 2,525.827526
32,OK Orion 1,525.939881


In [538]:
teams_to_follow = estimates[estimates['team_1'].str.contains("Reak") | estimates['team_1'].str.contains("Puskasil") | estimates['team_1'].str.contains("Rastihaukat")]
teams_to_follow[["team_1", "fin_median_7", "fin_start95_7", "fin_end95_7"]]

,team_1,fin_median_7,fin_start95_7,fin_end95_7
team_id,,,,
429,Reaktor Innovations 1,744.906260,698.688113,796.902706
568,Rastihaukat 1,751.586898,698.912018,809.994591
1270,Reaktor Innovations 2,884.872947,793.519981,991.197181
1246,Rastihaukat 2,909.786347,832.835051,996.400842
1131,Puskasilimät OK 1,913.017857,855.307102,976.898347


In [530]:
leg_1_cols = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], leg_1_cols))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_").sort_values(by=['team_id', 'leg'])
runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
runner_estimates.head()

team  leg_dist                  name  history_log_means  \
team_id leg                                                                   
5       1    Halden SK 1      11.0        Emil Wingstedt           1.763072   
        2    Halden SK 1      11.9      Fredrik Eliasson           1.842846   
        3    Halden SK 1      12.8  Thomas Natvig Årstad           1.797910   
        4    Halden SK 1       8.7    Alexander Chepelin           1.929038   
        5    Halden SK 1       8.7       Riccardo Scalet           1.958229   

             history_log_stdevs  log_means  log_stdevs  ind_95_start  \
team_id leg                                                            
5       1              0.025436   1.763072    0.025436     61.014607   
        2              0.093588   1.842846    0.093588     62.549244   
        3              0.103600   1.797910    0.103600     63.073600   
        4              0.069687   1.929038    0.069687     52.236299   
        5              0.119586   1.958229    0.119586     48.772555   

             ind_95_end   ind_mean  ind_median  fin_start95   fin_end95  \
team_id leg                                                               
5       1     67.411824  64.154251   64.133501    61.096118   67.378716   
        2     90.270797  75.472132   75.142332   126.038682  155.070741   
        3     94.671024  77.689619   77.273814   197.366621  239.496804   
        4     68.644760  60.026693   59.881118   255.745094  301.101096   
        5     77.939788  62.097304   61.654866   313.437920  367.719367   

             fin_median    fin_mean                fint_median  \
team_id leg                                                      
5       1     64.149176   64.164100 2018-06-17 00:04:08.950545   
        2    139.376193  139.697916 2018-06-17 01:19:22.571584   
        3    217.172339  217.472672 2018-06-17 02:37:10.340341   
        4    277.237239  277.457955 2018-06-17 03:37:14.234323   
        5    339.197981  339.525671 2018-06-17 04:39:11.878851   

                          fint_start95                    fint_end95  
team_id leg                                                           
5       1   2018-06-17 00:01:05.767089 2018-06-17 00:07:22.722948000  
        2   2018-06-17 01:06:02.320944 2018-06-17 01:35:04.244449000  
        3   2018-06-17 02:17:21.997240 2018-06-17 02:59:29.808250000  
        4   2018-06-17 03:15:44.705657 2018-06-17 04:01:06.065741999  
        5   2018-06-17 04:13:26.275222 2018-06-17 05:07:43.162004000

In [531]:
runner_estimates.to_csv('data/runner_estimates_ju2018.tsv', sep="\t")

In [532]:
for_print = runner_estimates.copy()
for_print = for_print.reset_index()
for_print = for_print.set_index('team_id')
for_print = for_print.round(1)
for_print.fint_median = for_print.fint_median.dt.strftime("%H:%M")
for_print.fint_start95 = for_print.fint_start95.dt.strftime("%H:%M")
for_print.fint_end95 = for_print.fint_end95.dt.strftime("%H:%M")
for_print = for_print[for_print['team'].str.contains("Reak") | for_print['team'].str.contains("Puskasil") | for_print['team'].str.contains("Rastihaukat 2")]
for_print = for_print[['team',
 'leg',
 'name',
 'ind_median',
 'fin_median',
 'fint_median',
 'fint_start95',
 'fint_end95']]
for_print

,team,leg,name,ind_median,fin_median,fint_median,fint_start95,fint_end95
team_id,,,,,,,,
429,Reaktor Innovations 1,1,Mikko Peltonen,97.8,97.6,00:37,00:23,00:54
429,Reaktor Innovations 1,2,Pasi Huhtiniemi,109.8,207.8,02:27,02:06,02:50
429,Reaktor Innovations 1,3,Joni Freeman,107.8,316.1,04:16,03:45,04:51
429,Reaktor Innovations 1,4,Tuomas Kareinen,86.2,403.0,05:42,05:03,06:29
429,Reaktor Innovations 1,5,Janne Vaittinen,88.9,492.1,07:12,06:31,07:58
429,Reaktor Innovations 1,6,Olavi Kanerva,90.8,583.6,08:43,08:00,09:33
429,Reaktor Innovations 1,7,Jyri Kytömäki,161.3,744.9,11:24,10:38,12:16
1131,Puskasilimät OK 1,1,Tuomas Ketonen,124.3,124.4,01:04,00:41,01:32
1131,Puskasilimät OK 1,2,Petri Miettinen,131.1,255.7,03:15,02:50,03:45


In [533]:
for_print.to_csv('data/for_print_ju2018.tsv', sep="\t")